In [13]:
from hdr.EfficientNet import *
from hdr.preprocess import *

# Data Prepration.
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

import copy
import os
import pandas as pd

In [14]:
# constants 

# version: width, depth, res, dropout rate
efficient_net_config = {
"b0" : (1.0, 1.0, 224, 0.2),
"b1" : (1.0, 1.1, 240, 0.2),
"b2" : (1.1, 1.2, 260, 0.3),
"b3" : (1.2, 1.4, 300, 0.3),
"b4" : (1.4, 1.8, 380, 0.4),
"b5" : (1.6, 2.2, 456, 0.4),
"b6" : (1.8, 2.6, 528, 0.5),
"b7" : (2.0, 3.1, 600, 0.5)
}

version = 'b0'
width_mult, depth_mult, res, dropout_rate = efficient_net_config[version]


In [16]:
DATA_DIR = os.getcwd() + "//resources"  
net = EfficientNet(width_mult, depth_mult, dropout_rate)


In [17]:
# Reading the labels.
print(f"\nTraining labels stored in [{os.listdir(DATA_DIR + "//train")[1]}]. \n >>If this file is incorrect, ensure only 1 file labelled [trainLabels.csv]]!!")

try:
    trainlabel = pd.read_csv(DATA_DIR + "//train//" + os.listdir(DATA_DIR + "//train")[1])
except Exception as e:
    print("\nTraining Labels cannot be loaded", e)

# load in files
trainimages = os.listdir(DATA_DIR + "//data//sample")
# remove the ".jpeg"
trainimages_nojpeg = [x.split(".")[0] for x in trainimages]
# get all current training images
trainlabel_F =  trainlabel[trainlabel['image'].isin(trainimages_nojpeg)]

print(f"\nTraining set consists of {len(trainlabel_F)} images.")


Training labels stored in [trainLabels.csv]. 
 >>If this file is incorrect, ensure only 1 file labelled [trainLabels.csv]]!!

Training set consists of 10 images.


In [19]:
# Dataset and Dataloaders.
# Hyper-params
BATCH_SIZE = 32
NUM_OF_CLASSES = 5
device = torch.device('cuda')

# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


# Datasets
train_dataset = DiabeticRetinopathyDataset(DATA_DIR + "//data//sample", data_transforms['train'])
val_dataset = DiabeticRetinopathyDataset(DATA_DIR + "//data//sample", data_transforms['val'])


# Datloaders
train_loader = DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE, shuffle = True)
val_loader = DataLoader(dataset = val_dataset, batch_size = BATCH_SIZE, shuffle = False)


In [ ]:
# Model Performance on test data
def calculate_loss_and_accuracy(model, dataloader, size_of_dataset, criterion, device):
    
    # Now set model to validation mode.
    running_loss = 0
    running_accuracy = 0
    
     # Processing the Test Loader
    for (inputs, labels) in dataloader:
        
        # Load data to device.
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Outputs
        outputs = model(inputs)
        _ , preds = torch.max(outputs, 1)
        
        # Outputs
        outputs = model(inputs)
        _ , preds = torch.max(outputs, 1)
        
        # Loss and Backpropagation.
        loss = criterion(outputs, labels)
        
        # Statistics
        running_loss += loss.item()*inputs.size(0)
        running_accuracy += torch.sum(preds == labels.data)
        
    epoch_loss = running_loss/size_of_dataset
    epoch_accuracy = running_accuracy/size_of_dataset
    
    return epoch_loss, epoch_accuracy

#------------------------------------------------------------------------------------------------
def train(model, criterion, optimizer, scheduler, num_of_epochs, device, train_loader):
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    #track_training_loss = []
    #track_val_loss = []

    for epoch in range(num_of_epochs):

        print(f'\nEpoch {epoch + 1}/{num_of_epochs}')
        print('-'*30)

        model.train() # Setting model to train.
        running_loss = 0
        running_accuracy = 0

        # Processing the Train Loader
        for (inputs, labels) in train_loader:

            # Load data to device.
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad() # zero the parameter gradients

            # Outputs
            outputs = model(inputs)
            _ , preds = torch.max(outputs, 1)

            # Loss and Backpropagation.
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item()*inputs.size(0)
            running_accuracy += torch.sum(preds == labels.data)
        
        scheduler.step()
        epoch_loss = running_loss/len(train_dataset)
        epoch_accuracy = running_accuracy/len(train_dataset)
        #track_training_loss.append(epoch_loss) # Loss Tracking

        print(f'Training Loss: {epoch_loss:.4f} Training Acc.: {epoch_accuracy:.4f}')

        # Now set model to validation mode.
        model.eval()

        val_loss, val_accuracy = calculate_loss_and_accuracy(model, val_loader, len(val_dataset), criterion)

        if val_accuracy > best_acc:
            print("Found better model...")
            print('Updating the model weights....\n')
            print(f'Val Loss: {val_loss:.4f} Val Acc.: {val_accuracy:.4f}\n')

            best_acc = val_accuracy
            best_model_wts = copy.deepcopy(model.state_dict())
     
    model.load_state_dict(best_model_wts) # update model
    
    return  model